In [1]:
# Import necessary libraries
!pip install mediapipe
import mediapipe as mp 
import numpy as np 
import cv2 

DEPRECATION: mermaid 0.3.2 has a non-standard dependency specifier torch>=1.7torchvision. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mermaid or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
    --------------------------------------- 1.0/50.8 MB 20.7 MB/s eta 0:00:03
   -- ------------------------------------- 2.7/50.8 MB 35.1 MB/s eta 0:00:02
   --- ------------------------------------ 3.9/50.8 MB 28.0 MB/s eta 0:00:02
   --- ------------------------------------ 4.5/50.8 MB 26.0 MB/s eta 0:00:02
   ---- ----------------------------------- 5.4/50.8 MB 24.8 MB/s eta 0:00:02
   ---- ----------------------------------- 5.9/50.8 MB 22.3 MB/s eta 0:00:03
   ---- ----------------------------------- 6.2/50.8 MB 18.8 MB/s eta 0:00:03
   ----- ---------------------------------- 6.5/50.8 MB 18.0 MB/s eta 0:00:03
   ----- ---------------------------------- 6.8/50.8 MB 16.1 MB/s eta 0:00:03
   ----- ---------------------------------- 7.2/50.8 MB 15.9 MB/s eta 0:00:03
   ----- ---------------------------------- 7.6/50.8 MB 15.2 MB/s eta 0:00:03
   ------ --------------------------------- 8.0/50.8 MB 14.3 MB/s eta 0:

In [2]:
# Open webcam for video capture
cap = cv2.VideoCapture(0)


In [ ]:
# Ask user to input a name for the dataset
name = input("Enter the name of the data : ")

In [ ]:
# Initialize Mediapipe modules for holistic and hand detection
holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

In [ ]:
# Initialize variables for storing data
X = []
data_size = 0

In [ ]:
# Loop for capturing and processing video frames
while True:
    lst = []
    # Read a frame from the webcam
    _, frm = cap.read()
    # Flip the frame horizontally for better visualization
    frm = cv2.flip(frm, 1)
    # Process the frame using Mediapipe
    res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    # Extract facial and hand landmarks and calculate relative positions
    if res.face_landmarks:
        for i in res.face_landmarks.landmark:
            lst.append(i.x - res.face_landmarks.landmark[1].x)
            lst.append(i.y - res.face_landmarks.landmark[1].y)
    # Process left hand landmarks
    if res.left_hand_landmarks:
        for i in res.left_hand_landmarks.landmark:
            lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
    else:
        # If no hand landmarks detected, fill with zeros
        for i in range(42):
            lst.append(0.0)
    # Process right hand landmarks
    if res.right_hand_landmarks:
        for i in res.right_hand_landmarks.landmark:
            lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
    else:
        # If no hand landmarks detected, fill with zeros
        for i in range(42):
            lst.append(0.0)
    # Append the processed data to the dataset
    X.append(lst)
    data_size += 1


In [ ]:
# Draw landmarks on the frame for visualization
 drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_CONTOURS)
 drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
 drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)

In [ ]:
# Display the current data size on the frame
 cv2.putText(frm, str(data_size), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [ ]:
 # Display the frame
 cv2.imshow("window", frm)
 # Exit the loop if 'Esc' key is pressed or data size reaches 100
 if cv2.waitKey(1) == 27 or data_size > 99:
 cv2.destroyAllWindows()
 cap.release()
 break

In [ ]:
# Save the collected data to a numpy file
np.save(f"{name}.npy", np.array(X))
print(np.array(X).shape)

In [ ]:
# Install TensorFlow for model training
!pip install tensorflow

In [ ]:
# Import required libraries for model training
import os 
import numpy as np 
import cv2 
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense 
from keras.models import Model

In [ ]:
# Initialize variables for data processing
is_init = False
size = -1
label = []
dictionary = {}
c = 0

In [ ]:
# Loop through files in the directory for data processing
for i in os.listdir():
    # Check if file is a numpy file containing data
    if i.split(".")[-1] == "npy" and not(i.split(".")[0] == "labels"): 
        # Check if it's the first file encountered
        if not(is_init):
            is_init = True 
            X = np.load(i)
            size = X.shape[0]
            y = np.array([i.split('.')[0]] * size).reshape(-1, 1)
        else:
            # Concatenate data with previously loaded data
            X = np.concatenate((X, np.load(i)))
            y = np.concatenate((y, np.array([i.split('.')[0]] * size).reshape(-1, 1)))
        # Store label information
        label.append(i.split('.')[0])
        dictionary[i.split('.')[0]] = c 
        c += 1

In [ ]:
# Convert labels to numerical categorical values
for i in range(y.shape[0]):
    y[i, 0] = dictionary[y[i, 0]]

y = np.array(y, dtype="int32")
y = to_categorical(y)

In [ ]:
# Shuffle data and labels
X_new = X.copy()
y_new = y.copy()
counter = 0 
cnt = np.arange(X.shape[0])
np.random.shuffle(cnt)

for i in cnt: 
    X_new[counter] = X[i]
    y_new[counter] = y[i]
    counter += 1

In [ ]:
# Define neural network architecture
ip = Input(shape=(X.shape[1]))
m = Dense(512, activation="relu")(ip)
m = Dense(256, activation="relu")(m)
op = Dense(y.shape[1], activation="softmax")(m) 
model = Model(inputs=ip, outputs=op)


In [ ]:
# Compile and train the model
model.compile(optimizer='rmsprop', loss="categorical_crossentropy", 
metrics=['acc'])
model.fit(X, y, epochs=50)

In [ ]:
# Save the trained model and labels
model.save("model.h5")
np.save("labels.npy", np.array(label))

In [ ]:
# Load the trained model and labels for real-time emotion detection
import cv2 
import numpy as np 
import mediapipe as mp 
from keras.models import load_model

In [ ]:
model = load_model("model.h5")
label = np.load("labels.npy")

In [ ]:
# Initialize Mediapipe modules for real-time processing
holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    lst = []
    # Read a frame from the webcam
    _, frm = cap.read()
    frm = cv2.flip(frm, 1)
    # Process the frame using Mediapipe
    res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    # Extract facial and hand landmarks and calculate relative positions
    if res.face_landmarks:
        for i in res.face_landmarks.landmark:
            lst.append(i.x - res.face_landmarks.landmark[1].x)
            lst.append(i.y - res.face_landmarks.landmark[1].y)
    # Process left hand landmarks
    if res.left_hand_landmarks:
        for i in res.left_hand_landmarks.landmark:
            lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
    else:
        for i in range(42):
            lst.append(0.0)
    # Process right hand landmarks
    if res.right_hand_landmarks:
        for i in res.right_hand_landmarks.landmark:
            lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
            lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
    else:
        for i in range(42):
            lst.append(0.0)
    # Make prediction using the trained model
    lst = np.array(lst).reshape(1, -1)
    pred = label[np.argmax(model.predict(lst))]
    print(pred)
    # Display predicted emotion on the frame
    cv2.putText(frm, pred, (50, 50), cv2.FONT_ITALIC, 1, (255, 0, 0), 2)
    # Draw landmarks on the frame for visualization
    drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_CONTOURS)
    drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
    drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)
    # Display the frame
    cv2.imshow("window", frm)
    # Exit the loop if 'Esc' key is pressed
    if cv2.waitKey(1) == 27:
        cv2.destroyAllWindows()
        cap.release()
        break


In [ ]:
# Install required libraries for web application development
!pip install streamlit
!pip install streamlit-webrtc

In [ ]:
# Run the Streamlit application for music recommendation
!streamlit run music.py